# 챕터 9: OpenAI API로 Skills 파이프라인 자동화 (Pydantic 활용)

## 9-1. 개요

이번 챕터에서는 OpenAI API를 사용하여 Skills 파이프라인을 구축한다.
Pydantic을 활용하여 데이터 검증과 타입 안정성을 강화한다.

**주요 특징:**
- OpenAI GPT-4o-mini 모델 사용
- Pydantic v2로 강력한 데이터 검증
- 구조화된 출력 (Structured Output)
- 타입 안정성 보장
- 자동 문서화

**Pydantic의 장점:**
1. 자동 데이터 검증
2. 타입 힌트 기반 설계
3. JSON 스키마 자동 생성
4. IDE 자동완성 지원
5. 명확한 에러 메시지

In [ ]:
# 필요한 라이브러리 설치
%pip install openai pydantic python-dotenv pyyaml -q

In [26]:
from dotenv import load_dotenv
import os

MODEL = "gpt-4o-mini"
load_dotenv()

True

In [27]:
import openai
import os
from pathlib import Path
import yaml
import json
from typing import Dict, List, Optional, Any, Union, Literal
from datetime import datetime
import time

# Pydantic v2 임포트
from pydantic import BaseModel, Field, validator, ConfigDict
from pydantic import ValidationError

# OpenAI 클라이언트 초기화
client = openai.OpenAI()

print("환경 설정 완료")
print(f"OpenAI 라이브러리 버전: {openai.__version__}")

환경 설정 완료
OpenAI 라이브러리 버전: 2.6.1


## 9-2. Pydantic 모델 정의

Skill과 파이프라인 구성 요소를 Pydantic 모델로 정의한다.

In [28]:
class SkillConfig(BaseModel):
    """
    Skill의 설정을 정의하는 모델
    """
    model_config = ConfigDict(extra='forbid')  # 추가 필드 허용 안함
    
    model: str = Field(
        default=MODEL,
        description="사용할 OpenAI 모델"
    )
    max_tokens: int = Field(
        default=2000,
        ge=1,
        le=16000,
        description="최대 토큰 수"
    )
    temperature: float = Field(
        default=0.7,
        ge=0.0,
        le=2.0,
        description="온도 파라미터"
    )
    top_p: Optional[float] = Field(
        default=None,
        ge=0.0,
        le=1.0,
        description="Top-p 샘플링"
    )

class SkillMetadata(BaseModel):
    """
    Skill의 메타데이터
    """
    name: str = Field(..., description="Skill 이름")
    version: str = Field(default="1.0.0", description="버전")
    description: str = Field(..., description="설명")
    author: Optional[str] = Field(default=None, description="작성자")
    tags: List[str] = Field(default_factory=list, description="태그")
    created_at: Optional[datetime] = Field(
        default_factory=datetime.now,
        description="생성 시간"
    )

class Skill(BaseModel):
    """
    완전한 Skill 정의
    """
    metadata: SkillMetadata = Field(..., description="메타데이터")
    content: str = Field(..., description="Skill 프롬프트 내용")
    config: SkillConfig = Field(
        default_factory=SkillConfig,
        description="설정"
    )
    
    def to_system_message(self) -> str:
        """시스템 메시지로 변환한다"""
        return self.content
    
    @classmethod
    def from_yaml(cls, file_path: str) -> 'Skill':
        """
        YAML 파일에서 Skill을 로드한다
        
        Args:
            file_path: YAML 파일 경로
            
        Returns:
            Skill 인스턴스
        """
        with open(file_path, 'r', encoding='utf-8') as f:
            data = yaml.safe_load(f)
        
        # 데이터 구조 변환
        metadata = SkillMetadata(
            name=data['name'],
            version=data.get('version', '1.0.0'),
            description=data['description'],
            author=data.get('author'),
            tags=data.get('tags', [])
        )
        
        config_data = data.get('config', {})
        config = SkillConfig(**config_data)
        
        return cls(
            metadata=metadata,
            content=data['content'],
            config=config
        )
    
    def save_yaml(self, file_path: str) -> None:
        """
        Skill을 YAML 파일로 저장한다
        
        Args:
            file_path: 저장할 파일 경로
        """
        data = {
            'name': self.metadata.name,
            'version': self.metadata.version,
            'description': self.metadata.description,
            'author': self.metadata.author,
            'tags': self.metadata.tags,
            'content': self.content,
            'config': self.config.model_dump()
        }
        
        with open(file_path, 'w', encoding='utf-8') as f:
            yaml.dump(data, f, allow_unicode=True, default_flow_style=False)

print("Pydantic 모델 정의 완료")

Pydantic 모델 정의 완료


## 9-3. 파이프라인 구성 요소 모델

In [29]:
class PipelineStepInput(BaseModel):
    """
    파이프라인 단계의 입력 정의
    """
    context_keys: List[str] = Field(
        ...,
        description="컨텍스트에서 가져올 키 목록"
    )
    prompt_template: str = Field(
        ...,
        description="프롬프트 템플릿"
    )

class PipelineStepOutput(BaseModel):
    """
    파이프라인 단계의 출력 정의
    """
    key: str = Field(
        ...,
        description="결과를 저장할 컨텍스트 키"
    )
    validation_schema: Optional[Dict[str, Any]] = Field(
        default=None,
        description="출력 검증 스키마 (선택)"
    )

class PipelineStep(BaseModel):
    """
    파이프라인의 단일 단계
    """
    name: str = Field(..., description="단계 이름")
    skill_name: str = Field(..., description="사용할 Skill 이름")
    input: PipelineStepInput = Field(..., description="입력 정의")
    output: PipelineStepOutput = Field(..., description="출력 정의")
    description: Optional[str] = Field(
        default=None,
        description="단계 설명"
    )
    retry_on_error: bool = Field(
        default=False,
        description="에러 시 재시도 여부"
    )
    max_retries: int = Field(
        default=3,
        ge=0,
        description="최대 재시도 횟수"
    )

class PipelineContext(BaseModel):
    """
    파이프라인 실행 컨텍스트
    """
    model_config = ConfigDict(arbitrary_types_allowed=True)
    
    data: Dict[str, Any] = Field(
        default_factory=dict,
        description="파이프라인 데이터"
    )
    metadata: Dict[str, Any] = Field(
        default_factory=dict,
        description="메타데이터"
    )
    
    def set(self, key: str, value: Any) -> None:
        """데이터를 설정한다"""
        self.data[key] = value
    
    def get(self, key: str, default: Any = None) -> Any:
        """데이터를 가져온다"""
        return self.data.get(key, default)
    
    def set_metadata(self, key: str, value: Any) -> None:
        """메타데이터를 설정한다"""
        self.metadata[key] = value
    
    def get_metadata(self, key: str, default: Any = None) -> Any:
        """메타데이터를 가져온다"""
        return self.metadata.get(key, default)

class PipelineExecutionLog(BaseModel):
    """
    파이프라인 실행 로그
    """
    step_name: str
    skill_name: str
    timestamp: datetime = Field(default_factory=datetime.now)
    elapsed_time: float
    input_length: int
    output_length: int
    success: bool
    error_message: Optional[str] = None
    retry_count: int = 0

print("파이프라인 구성 요소 모델 정의 완료")

파이프라인 구성 요소 모델 정의 완료


## 9-4. OpenAI 기반 파이프라인 엔진

In [35]:
class OpenAISkillPipeline:
    """
    OpenAI API를 사용하는 Skill 파이프라인
    """
    
    def __init__(self, api_key: Optional[str] = None):
        """
        파이프라인을 초기화한다
        
        Args:
            api_key: OpenAI API 키
        """
        self.client = openai.OpenAI()
        self.skills: Dict[str, Skill] = {}  # Skill 저장소
        self.steps: List[PipelineStep] = []  # 파이프라인 단계
        self.execution_logs: List[PipelineExecutionLog] = []  # 실행 로그
    
    def load_skill(self, skill_path: str) -> None:
        """
        Skill을 로드한다
        
        Args:
            skill_path: Skill 파일 경로
        """
        try:
            skill = Skill.from_yaml(skill_path)
            self.skills[skill.metadata.name] = skill
            print(f"Skill 로드: {skill.metadata.name} v{skill.metadata.version}")
        except ValidationError as e:
            print(f"Skill 로드 실패: {e}")
            raise
    
    def add_step(self, step: PipelineStep) -> None:
        """
        파이프라인에 단계를 추가한다
        
        Args:
            step: 추가할 파이프라인 단계
        """
        if step.skill_name not in self.skills:
            raise ValueError(
                f"Skill '{step.skill_name}'이 로드되지 않았다. "
                f"사용 가능한 Skills: {list(self.skills.keys())}"
            )
        
        self.steps.append(step)
        print(f"단계 추가: {step.name} (Skill: {step.skill_name})")
    
    def _create_prompt(
        self,
        step: PipelineStep,
        context: PipelineContext
    ) -> str:
        """
        컨텍스트 데이터로 프롬프트를 생성한다
        
        Args:
            step: 파이프라인 단계
            context: 파이프라인 컨텍스트
            
        Returns:
            생성된 프롬프트
        """
        # 컨텍스트에서 필요한 데이터 추출
        prompt_data = {
            key: context.get(key, '')
            for key in step.input.context_keys
        }
        
        # 템플릿에 데이터 삽입
        return step.input.prompt_template.format(**prompt_data)
    
    def _execute_step_once(
        self,
        step: PipelineStep,
        context: PipelineContext
    ) -> str:
        """
        단일 단계를 한 번 실행한다
        
        Args:
            step: 실행할 단계
            context: 파이프라인 컨텍스트
            
        Returns:
            실행 결과
        """
        skill = self.skills[step.skill_name]
        user_prompt = self._create_prompt(step, context)
        
        # OpenAI API 호출
        response = self.client.chat.completions.create(
            model=skill.config.model,
            max_tokens=skill.config.max_tokens,
            temperature=skill.config.temperature,
            top_p=skill.config.top_p,
            messages=[
                {
                    "role": "system",
                    "content": skill.to_system_message()
                },
                {
                    "role": "user",
                    "content": user_prompt
                }
            ]
        )
        
        return response.choices[0].message.content
    
    def execute_step(
        self,
        step: PipelineStep,
        context: PipelineContext
    ) -> str:
        """
        파이프라인 단계를 실행한다 (재시도 포함)
        
        Args:
            step: 실행할 단계
            context: 파이프라인 컨텍스트
            
        Returns:
            실행 결과
        """
        retry_count = 0
        last_error = None
        
        while retry_count <= step.max_retries:
            try:
                start_time = time.time()
                result = self._execute_step_once(step, context)
                elapsed_time = time.time() - start_time
                
                # 성공 로그
                log = PipelineExecutionLog(
                    step_name=step.name,
                    skill_name=step.skill_name,
                    elapsed_time=elapsed_time,
                    input_length=len(self._create_prompt(step, context)),
                    output_length=len(result),
                    success=True,
                    retry_count=retry_count
                )
                self.execution_logs.append(log)
                
                return result
                
            except Exception as e:
                last_error = str(e)
                retry_count += 1
                
                if retry_count <= step.max_retries and step.retry_on_error:
                    print(f"  재시도 {retry_count}/{step.max_retries}: {last_error}")
                    time.sleep(2 ** retry_count)  # 지수 백오프
                else:
                    # 실패 로그
                    log = PipelineExecutionLog(
                        step_name=step.name,
                        skill_name=step.skill_name,
                        elapsed_time=0,
                        input_length=0,
                        output_length=0,
                        success=False,
                        error_message=last_error,
                        retry_count=retry_count - 1
                    )
                    self.execution_logs.append(log)
                    raise
    
    def run(self, initial_data: Dict[str, Any]) -> PipelineContext:
        """
        파이프라인을 실행한다
        
        Args:
            initial_data: 초기 입력 데이터
            
        Returns:
            실행 완료된 컨텍스트
        """
        context = PipelineContext(data=initial_data.copy())
        context.set_metadata('start_time', datetime.now().isoformat())
        #context.set_metadata('model')
        
        print("\n" + "=" * 80)
        print("OpenAI 파이프라인 실행 시작")
        print(f"모델: gpt-4o-mini")
        print("=" * 80)
        
        for i, step in enumerate(self.steps, 1):
            print(f"\n[{i}/{len(self.steps)}] {step.name} 실행 중...")
            
            try:
                result = self.execute_step(step, context)
                context.set(step.output.key, result)
                print(f"  완료 (출력 길이: {len(result)} 문자)")
                
            except Exception as e:
                print(f"  오류 발생: {e}")
                context.set_metadata(f'error_{step.name}', str(e))
                raise
        
        context.set_metadata('end_time', datetime.now().isoformat())
        
        print("\n" + "=" * 80)
        print("파이프라인 실행 완료")
        print("=" * 80)
        
        return context
    
    def get_execution_summary(self) -> str:
        """
        실행 로그 요약을 반환한다
        
        Returns:
            실행 요약 문자열
        """
        if not self.execution_logs:
            return "실행 로그가 없다"
        
        summary = "\n파이프라인 실행 요약:\n"
        summary += "=" * 80 + "\n"
        
        total_time = sum(log.elapsed_time for log in self.execution_logs)
        success_count = sum(1 for log in self.execution_logs if log.success)
        
        for log in self.execution_logs:
            status = "성공" if log.success else "실패"
            summary += f"\n단계: {log.step_name} ({status})\n"
            summary += f"  Skill: {log.skill_name}\n"
            summary += f"  실행 시간: {log.elapsed_time:.2f}초\n"
            
            if log.success:
                summary += f"  입력 길이: {log.input_length} 문자\n"
                summary += f"  출력 길이: {log.output_length} 문자\n"
            else:
                summary += f"  에러: {log.error_message}\n"
            
            if log.retry_count > 0:
                summary += f"  재시도 횟수: {log.retry_count}\n"
        
        summary += f"\n총 실행 시간: {total_time:.2f}초\n"
        summary += f"성공/전체: {success_count}/{len(self.execution_logs)}\n"
        
        return summary

print("OpenAISkillPipeline 클래스 정의 완료")

OpenAISkillPipeline 클래스 정의 완료


## 9-5. 실습용 Skills 생성

In [36]:
# skills 디렉토리 생성
skills_dir = Path("skills/pipeline_openai")
skills_dir.mkdir(exist_ok=True)

# 1. 제품 아이디어 브레인스토밍 Skill
brainstorm_skill = Skill(
    metadata=SkillMetadata(
        name="product_brainstorm",
        version="1.0.0",
        description="제품 아이디어를 브레인스토밍하는 전문가",
        author="Tutorial",
        tags=["brainstorming", "product", "innovation"]
    ),
    content="""# 제품 브레인스토밍 전문가 Skill

## 역할
혁신적이고 실현 가능한 제품 아이디어를 생성하는 전문가다.

## 브레인스토밍 원칙
1. 발산적 사고: 다양한 관점에서 아이디어를 생성한다
2. 실용성: 실제 구현 가능한 아이디어를 우선한다
3. 차별화: 경쟁 제품과 구별되는 특징을 강조한다
4. 고객 중심: 타겟 고객의 문제를 해결한다

## 출력 형식
- 3-5개의 구체적인 아이디어를 제시한다
- 각 아이디어는 다음을 포함한다:
  * 제품명
  * 핵심 가치
  * 타겟 고객
  * 주요 기능 (3가지)
  * 차별화 포인트

## 스타일
- 문장은 ~다 체를 사용한다
- 열정적이고 긍정적인 톤을 유지한다
- 구체적인 예시를 포함한다
""",
    config=SkillConfig(
        model=MODEL,
        max_tokens=2500,
        temperature=0.9
    )
)

# 2. 시장 분석 Skill
market_analysis_skill = Skill(
    metadata=SkillMetadata(
        name="market_analyzer",
        version="1.0.0",
        description="시장과 경쟁 환경을 분석하는 전문가",
        author="Tutorial",
        tags=["market", "analysis", "competition"]
    ),
    content="""# 시장 분석 전문가 Skill

## 역할
시장 동향과 경쟁 환경을 체계적으로 분석하는 전문가다.

## 분석 프레임워크
1. 시장 규모 및 성장률
2. 타겟 고객 세분화
3. 경쟁사 분석 (3-5개)
4. 시장 트렌드
5. 진입 장벽 및 기회

## 분석 원칙
- 데이터 기반 분석을 수행한다
- SWOT 분석을 포함한다
- 정량적 지표를 제시한다
- 실행 가능한 인사이트를 도출한다

## 출력 형식
- 구조화된 분석 보고서 형태
- 섹션별로 명확히 구분한다
- 핵심 발견사항을 강조한다

## 스타일
- 문장은 ~다 체를 사용한다
- 전문적이고 객관적인 언어를 사용한다
- 구체적인 수치와 예시를 포함한다
""",
    config=SkillConfig(
        model=MODEL,
        max_tokens=3000,
        temperature=0.5
    )
)

# 3. 비즈니스 모델 설계 Skill
business_model_skill = Skill(
    metadata=SkillMetadata(
        name="business_model_designer",
        version="1.0.0",
        description="지속 가능한 비즈니스 모델을 설계하는 전문가",
        author="Tutorial",
        tags=["business", "model", "monetization"]
    ),
    content="""# 비즈니스 모델 설계 전문가 Skill

## 역할
제품의 수익화 전략과 비즈니스 모델을 설계하는 전문가다.

## 설계 프레임워크
Business Model Canvas 기반으로 다음을 포함한다:
1. 고객 세그먼트
2. 가치 제안
3. 채널
4. 고객 관계
5. 수익원
6. 핵심 자원
7. 핵심 활동
8. 핵심 파트너십
9. 비용 구조

## 설계 원칙
- 지속 가능성을 우선한다
- 확장 가능한 모델을 제안한다
- 다양한 수익원을 고려한다
- 실현 가능성을 검증한다

## 출력 형식
- Business Model Canvas 형태로 구조화한다
- 각 요소를 명확히 설명한다
- 예상 수익 모델을 제시한다

## 스타일
- 문장은 ~다 체를 사용한다
- 전략적이고 설득력 있는 언어를 사용한다
- 구체적인 수치와 시나리오를 포함한다
""",
    config=SkillConfig(
        model=MODEL,
        max_tokens=3500,
        temperature=0.6
    )
)

# 4. 실행 계획 수립 Skill
execution_plan_skill = Skill(
    metadata=SkillMetadata(
        name="execution_planner",
        version="1.0.0",
        description="구체적인 실행 계획을 수립하는 전문가",
        author="Tutorial",
        tags=["execution", "planning", "roadmap"]
    ),
    content="""# 실행 계획 전문가 Skill

## 역할
아이디어를 실행 가능한 단계별 계획으로 전환하는 전문가다.

## 계획 구성 요소
1. 마일스톤 정의 (6-12개월)
2. 단계별 액션 아이템
3. 필요 리소스 (인력, 예산, 기술)
4. 타임라인 및 우선순위
5. 성공 지표 (KPI)
6. 리스크 및 대응 방안

## 계획 수립 원칙
- SMART 목표를 설정한다
- 현실적이고 달성 가능한 계획을 수립한다
- 단계별 검증 포인트를 포함한다
- 유연성을 고려한다

## 출력 형식
- 타임라인 기반으로 구조화한다
- Gantt 차트 형태로 표현 가능하게 작성한다
- 각 단계의 소유자와 책임을 명시한다

## 스타일
- 문장은 ~다 체를 사용한다
- 명확하고 실행 지향적인 언어를 사용한다
- 구체적인 날짜와 수치를 포함한다
""",
    config=SkillConfig(
        model=MODEL,
        max_tokens=3000,
        temperature=0.4
    )
)

# Skills 저장
skills_to_save = [
    brainstorm_skill,
    market_analysis_skill,
    business_model_skill,
    execution_plan_skill
]

for skill in skills_to_save:
    skill_path = skills_dir / f"{skill.metadata.name}.yaml"
    skill.save_yaml(str(skill_path))
    print(f"Skill 저장: {skill_path}")

print(f"\n총 {len(skills_to_save)}개의 Skill이 생성되었다")

Skill 저장: skills/pipeline_openai/product_brainstorm.yaml
Skill 저장: skills/pipeline_openai/market_analyzer.yaml
Skill 저장: skills/pipeline_openai/business_model_designer.yaml
Skill 저장: skills/pipeline_openai/execution_planner.yaml

총 4개의 Skill이 생성되었다


## 9-6. 스타트업 아이디어 검증 파이프라인

In [37]:
# 파이프라인 생성
pipeline = OpenAISkillPipeline()

# Skills 로드
pipeline.load_skill('skills/pipeline_openai/product_brainstorm.yaml')
pipeline.load_skill('skills/pipeline_openai/market_analyzer.yaml')
pipeline.load_skill('skills/pipeline_openai/business_model_designer.yaml')
pipeline.load_skill('skills/pipeline_openai/execution_planner.yaml')

print("\n파이프라인 Skills 로드 완료")

Skill 로드: product_brainstorm v1.0.0
Skill 로드: market_analyzer v1.0.0
Skill 로드: business_model_designer v1.0.0
Skill 로드: execution_planner v1.0.0

파이프라인 Skills 로드 완료


In [38]:
# 파이프라인 단계 정의

# 1단계: 제품 아이디어 브레인스토밍
step1 = PipelineStep(
    name="제품 아이디어 브레인스토밍",
    skill_name="product_brainstorm",
    input=PipelineStepInput(
        context_keys=["domain", "target_audience"],
        prompt_template="""{domain} 분야에서 {target_audience}를 위한 혁신적인 제품 아이디어 3개를 제안해줘.
각 아이디어는 제품명, 핵심 가치, 타겟 고객, 주요 기능, 차별화 포인트를 포함해야 한다."""
    ),
    output=PipelineStepOutput(
        key="product_ideas"
    ),
    description="도메인과 타겟 고객을 기반으로 제품 아이디어를 생성한다",
    retry_on_error=True,
    max_retries=2
)

# 2단계: 시장 분석
step2 = PipelineStep(
    name="시장 분석",
    skill_name="market_analyzer",
    input=PipelineStepInput(
        context_keys=["product_ideas", "domain"],
        prompt_template="""다음 제품 아이디어들을 검토하고, {domain} 시장을 분석해줘:

{product_ideas}

시장 규모, 성장률, 경쟁사, 트렌드, 진입 장벽 및 기회를 포함한 종합적인 시장 분석을 제공해줘."""
    ),
    output=PipelineStepOutput(
        key="market_analysis"
    ),
    description="제품 아이디어가 타겟하는 시장을 분석한다",
    retry_on_error=True
)

# 3단계: 비즈니스 모델 설계
step3 = PipelineStep(
    name="비즈니스 모델 설계",
    skill_name="business_model_designer",
    input=PipelineStepInput(
        context_keys=["product_ideas", "market_analysis"],
        prompt_template="""다음 정보를 바탕으로 첫 번째 제품 아이디어에 대한 비즈니스 모델을 설계해줘:

제품 아이디어:
{product_ideas}

시장 분석:
{market_analysis}

Business Model Canvas 형태로 고객 세그먼트, 가치 제안, 채널, 수익원 등을 포함한 완전한 비즈니스 모델을 제시해줘."""
    ),
    output=PipelineStepOutput(
        key="business_model"
    ),
    description="선택된 아이디어의 비즈니스 모델을 설계한다"
)

# 4단계: 실행 계획 수립
step4 = PipelineStep(
    name="실행 계획 수립",
    skill_name="execution_planner",
    input=PipelineStepInput(
        context_keys=["product_ideas", "business_model"],
        prompt_template="""다음 정보를 바탕으로 6개월간의 구체적인 실행 계획을 수립해줘:

제품:
{product_ideas}

비즈니스 모델:
{business_model}

마일스톤, 액션 아이템, 필요 리소스, 타임라인, KPI, 리스크 관리를 포함한 실행 가능한 계획을 제시해줘."""
    ),
    output=PipelineStepOutput(
        key="execution_plan"
    ),
    description="제품 출시를 위한 실행 계획을 수립한다"
)

# 파이프라인에 단계 추가
pipeline.add_step(step1)
pipeline.add_step(step2)
pipeline.add_step(step3)
pipeline.add_step(step4)

print("\n파이프라인 구성 완료")

단계 추가: 제품 아이디어 브레인스토밍 (Skill: product_brainstorm)
단계 추가: 시장 분석 (Skill: market_analyzer)
단계 추가: 비즈니스 모델 설계 (Skill: business_model_designer)
단계 추가: 실행 계획 수립 (Skill: execution_planner)

파이프라인 구성 완료


In [39]:
# 파이프라인 실행
initial_data = {
    "domain": "건강 및 웰니스 테크",
    "target_audience": "바쁜 직장인들 (25-40세)"
}

result_context = pipeline.run(initial_data)

# 실행 요약 출력
print("\n" + pipeline.get_execution_summary())


OpenAI 파이프라인 실행 시작
모델: gpt-4o-mini

[1/4] 제품 아이디어 브레인스토밍 실행 중...
  완료 (출력 길이: 1152 문자)

[2/4] 시장 분석 실행 중...
  완료 (출력 길이: 1835 문자)

[3/4] 비즈니스 모델 설계 실행 중...
  완료 (출력 길이: 1912 문자)

[4/4] 실행 계획 수립 실행 중...
  완료 (출력 길이: 2075 문자)

파이프라인 실행 완료


파이프라인 실행 요약:

단계: 제품 아이디어 브레인스토밍 (성공)
  Skill: product_brainstorm
  실행 시간: 12.50초
  입력 길이: 112 문자
  출력 길이: 1152 문자

단계: 시장 분석 (성공)
  Skill: market_analyzer
  실행 시간: 26.78초
  입력 길이: 1251 문자
  출력 길이: 1835 문자

단계: 비즈니스 모델 설계 (성공)
  Skill: business_model_designer
  실행 시간: 17.23초
  입력 길이: 3127 문자
  출력 길이: 1912 문자

단계: 실행 계획 수립 (성공)
  Skill: execution_planner
  실행 시간: 21.44초
  입력 길이: 3179 문자
  출력 길이: 2075 문자

총 실행 시간: 77.95초
성공/전체: 4/4



## 9-7. 결과 확인 및 저장

In [40]:
# 각 단계의 결과 미리보기
print("\n" + "=" * 80)
print("단계별 결과 미리보기")
print("=" * 80)

steps_output = [
    ("제품 아이디어", "product_ideas"),
    ("시장 분석", "market_analysis"),
    ("비즈니스 모델", "business_model"),
    ("실행 계획", "execution_plan")
]

for title, key in steps_output:
    print(f"\n[{title}]")
    content = result_context.get(key, "")
    preview = content[:300] + "..." if len(content) > 300 else content
    print(preview)


단계별 결과 미리보기

[제품 아이디어]
물론이다! 바쁜 직장인들을 위한 건강 및 웰니스 테크 분야의 혁신적인 제품 아이디어를 아래와 같이 제안한다.

### 1. 제품명: "WellSync 스마트 팔찌"
- **핵심 가치**: 개인 맞춤형 건강 관리 및 스트레스 완화
- **타겟 고객**: 25-40세 바쁜 직장인
- **주요 기능**:
  1. 실시간 심박수 및 스트레스 모니터링: 사용자의 심박수와 스트레스 수치를 분석하여 경고 신호 제공
  2. 맞춤형 운동 및 명상 프로그램: 사용자 데이터를 기반으로 개인화된 운동 및 명상 추천
  3. 휴식 알림 기능: 일정 시간...

[시장 분석]
# 건강 및 웰니스 테크 시장 분석 보고서

## 1. 시장 규모 및 성장률
2023년 건강 및 웰니스 테크 시장 규모는 약 4,500억 달러로 추정되며, 2028년까지 연평균 성장률(CAGR) 20% 이상을 기록할 것으로 예상된다. 이는 바쁜 직장인들의 건강 관리에 대한 관심 증가와 기술 발전이 주요 요인으로 작용하고 있다. 특히, COVID-19 팬데믹 이후 개인의 건강과 웰니스에 대한 인식이 높아지면서 해당 시장의 성장세가 가속화되고 있다.

## 2. 타겟 고객 세분화
- **연령대**: 25-40세
- **직업**: 바쁜 ...

[비즈니스 모델]
다음은 "WellSync 스마트 팔찌", "NutriTrack 식단 앱", "MindFlow VR 명상 키트" 제품 아이디어를 바탕으로 한 비즈니스 모델 캔버스이다.

### Business Model Canvas

#### 1. 고객 세그먼트
- **주요 고객**: 25-40세 바쁜 직장인
- **하위 세분화**:
  - 건강과 웰빙에 관심이 많은 중상위 소득층
  - 스트레스 관리와 정신적 웰빙을 중시하는 개인
  - 기술 수용도가 높은 사용자

#### 2. 가치 제안
- **WellSync 스마트 팔찌**: 개인 맞춤형 건강...

[실행 계획]
다음은 "WellSync 스마트 팔찌", "NutriTrack 

In [41]:
# 결과를 파일로 저장
def save_openai_pipeline_results(
    context: PipelineContext,
    logs: List[PipelineExecutionLog],
    output_dir: str = "openai_pipeline_output"
) -> None:
    """
    OpenAI 파이프라인 결과를 저장한다
    
    Args:
        context: 파이프라인 컨텍스트
        logs: 실행 로그
        output_dir: 출력 디렉토리
    """
    output_path = Path(output_dir)
    output_path.mkdir(exist_ok=True)
    
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    run_dir = output_path / f"run_{timestamp}"
    run_dir.mkdir(exist_ok=True)
    
    # 각 단계 결과를 개별 파일로 저장
    for key, value in context.data.items():
        if isinstance(value, str):
            file_path = run_dir / f"{key}.txt"
            with open(file_path, 'w', encoding='utf-8') as f:
                f.write(value)
            print(f"저장: {file_path}")
    
    # 메타데이터 저장 (Pydantic 모델 직렬화)
    metadata_path = run_dir / "metadata.json"
    with open(metadata_path, 'w', encoding='utf-8') as f:
        json.dump(context.metadata, f, ensure_ascii=False, indent=2, default=str)
    print(f"메타데이터 저장: {metadata_path}")
    
    # 실행 로그 저장
    logs_path = run_dir / "execution_logs.json"
    with open(logs_path, 'w', encoding='utf-8') as f:
        logs_data = [log.model_dump(mode='json') for log in logs]
        json.dump(logs_data, f, ensure_ascii=False, indent=2, default=str)
    print(f"실행 로그 저장: {logs_path}")
    
    # 종합 보고서 (Markdown)
    report_path = run_dir / "startup_validation_report.md"
    with open(report_path, 'w', encoding='utf-8') as f:
        f.write("# 스타트업 아이디어 검증 보고서\n\n")
        f.write(f"생성 시간: {timestamp}\n")
        f.write(f"모델: {context.get_metadata('model', 'gpt-4o-mini')}\n\n")
        
        f.write("## 입력 정보\n\n")
        f.write(f"- 도메인: {context.data.get('domain', 'N/A')}\n")
        f.write(f"- 타겟 고객: {context.data.get('target_audience', 'N/A')}\n\n")
        
        sections = [
            ("제품 아이디어", "product_ideas"),
            ("시장 분석", "market_analysis"),
            ("비즈니스 모델", "business_model"),
            ("실행 계획", "execution_plan")
        ]
        
        for title, key in sections:
            f.write(f"## {title}\n\n")
            f.write(context.get(key, "내용 없음"))
            f.write("\n\n---\n\n")
    
    print(f"종합 보고서 저장: {report_path}")
    print(f"\n모든 결과가 {run_dir}에 저장되었다")

# 결과 저장
save_openai_pipeline_results(result_context, pipeline.execution_logs)

저장: openai_pipeline_output/run_20251108_163720/domain.txt
저장: openai_pipeline_output/run_20251108_163720/target_audience.txt
저장: openai_pipeline_output/run_20251108_163720/product_ideas.txt
저장: openai_pipeline_output/run_20251108_163720/market_analysis.txt
저장: openai_pipeline_output/run_20251108_163720/business_model.txt
저장: openai_pipeline_output/run_20251108_163720/execution_plan.txt
메타데이터 저장: openai_pipeline_output/run_20251108_163720/metadata.json
실행 로그 저장: openai_pipeline_output/run_20251108_163720/execution_logs.json
종합 보고서 저장: openai_pipeline_output/run_20251108_163720/startup_validation_report.md

모든 결과가 openai_pipeline_output/run_20251108_163720에 저장되었다


## 9-8. Pydantic 검증 예제

Pydantic의 강력한 검증 기능을 활용한 예제다.

In [42]:
# 구조화된 출력을 위한 Pydantic 모델
class ProductIdea(BaseModel):
    """제품 아이디어 구조화"""
    name: str = Field(..., description="제품명")
    tagline: str = Field(..., description="한 줄 설명")
    target_customer: str = Field(..., description="타겟 고객")
    core_features: List[str] = Field(
        ...,
        min_length=3,
        max_length=5,
        description="핵심 기능"
    )
    differentiation: str = Field(..., description="차별화 포인트")
    estimated_market_size: Optional[str] = Field(
        default=None,
        description="예상 시장 규모"
    )

class ProductIdeaCollection(BaseModel):
    """여러 제품 아이디어 모음"""
    ideas: List[ProductIdea] = Field(
        ...,
        min_length=1,
        max_length=5
    )
    domain: str
    generated_at: datetime = Field(default_factory=datetime.now)

# 검증 예제
try:
    # 올바른 데이터
    valid_idea = ProductIdea(
        name="헬스케어 AI 어시스턴트",
        tagline="AI가 관리하는 개인 맞춤 건강 비서",
        target_customer="바쁜 직장인 (25-40세)",
        core_features=[
            "실시간 건강 모니터링",
            "AI 기반 건강 조언",
            "운동 및 식단 추천"
        ],
        differentiation="개인화된 AI 코칭"
    )
    print("검증 성공:")
    print(valid_idea.model_dump_json(indent=2))
    
except ValidationError as e:
    print("검증 실패:")
    print(e)

print("\n" + "=" * 80)

# 잘못된 데이터 예제
try:
    invalid_idea = ProductIdea(
        name="테스트",
        tagline="설명",
        target_customer="고객",
        core_features=["기능1"],  # 최소 3개 필요
        differentiation="차별화"
    )
except ValidationError as e:
    print("예상된 검증 오류:")
    print(e)

검증 성공:
{
  "name": "헬스케어 AI 어시스턴트",
  "tagline": "AI가 관리하는 개인 맞춤 건강 비서",
  "target_customer": "바쁜 직장인 (25-40세)",
  "core_features": [
    "실시간 건강 모니터링",
    "AI 기반 건강 조언",
    "운동 및 식단 추천"
  ],
  "differentiation": "개인화된 AI 코칭",
  "estimated_market_size": null
}

예상된 검증 오류:
1 validation error for ProductIdea
core_features
  List should have at least 3 items after validation, not 1 [type=too_short, input_value=['기능1'], input_type=list]
    For further information visit https://errors.pydantic.dev/2.11/v/too_short


## 9-9. 생성된 파일 목록

In [43]:
print("\n" + "=" * 80)
print("생성된 모든 파일")
print("=" * 80)

# OpenAI Skills
print("\n[OpenAI Skills]")
for file_path in sorted(skills_dir.glob("*.yaml")):
    size = file_path.stat().st_size
    print(f"  {file_path.name} ({size:,} bytes)")

# 파이프라인 출력
output_dir = Path("openai_pipeline_output")
if output_dir.exists():
    print("\n[파이프라인 실행 결과]")
    for run_dir in sorted(output_dir.glob("run_*")):
        print(f"  {run_dir.name}/")
        for file_path in sorted(run_dir.glob("*")):
            size = file_path.stat().st_size
            print(f"    {file_path.name} ({size:,} bytes)")


생성된 모든 파일

[OpenAI Skills]
  business_model_designer.yaml (1,205 bytes)
  execution_planner.yaml (1,188 bytes)
  market_analyzer.yaml (1,057 bytes)
  product_brainstorm.yaml (1,101 bytes)

[파이프라인 실행 결과]
  run_20251108_163720/
    business_model.txt (3,768 bytes)
    domain.txt (27 bytes)
    execution_logs.json (1,214 bytes)
    execution_plan.txt (3,651 bytes)
    market_analysis.txt (3,773 bytes)
    metadata.json (92 bytes)
    product_ideas.txt (2,494 bytes)
    startup_validation_report.md (14,004 bytes)
    target_audience.txt (30 bytes)
